In [1]:
#-*- coding: utf-8 -*-
import requests
from dotenv import load_dotenv
import os, sys, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px

from glob import glob
from tqdm.auto import tqdm

from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import ColumnDataSource, HoverTool, BoxAnnotation
from bokeh.palettes import Category10
from bokeh.layouts import layout
from bokeh.io.export import export_png

sys.path.append(os.path.abspath("../../code"))

import functions as fc

from naverApi_function import (
    TimeUnit,
    NaverAPIConfig,
    NaverShoppingInsight,
    CategoryRequestConfig, DeviceRequestConfig, GenderRequestConfig, AgeRequestConfig, KeywordRequestConfig, generate_monthly_dates, plot_keyword_age_trends
)

# pandas 설정
pd.set_option('display.max_columns', 100)

# matplotlib 설정
plt.rcParams['font.family'] = 'NanumGothic'  # 나눔고딕 설정
plt.rcParams['axes.unicode_minus'] = False   # 마이너스 기호 깨짐 방지

load_dotenv()

client_id = os.getenv("naver_client_id")
client_secret = os.getenv("naver_client_secret")

In [2]:
# API 설정
api_config = NaverAPIConfig(
    client_id=client_id,
    client_secret=client_secret
)

# API 클라이언트 생성
client = NaverShoppingInsight(api_config)

1주차 공급 데이터 분석에서 변동성이 적었던 도매 품목 리스트: \
['갈치', '피마늘', '포도', '양파', '새송이버섯', '브로콜리', '배', '딸기', '사과', '단감', '고구마', '굴', '레몬', '망고', '감자', '느타리버섯', '당근', '감귤', '토마토', '체리']

---

🥗 1. 브로콜리 고구마 닭가슴살 볼
	•	브로콜리, 고구마, 양파, 레몬 드레싱
	•	👉 저탄수 + 고단백 식단, 렌지 3분 조리 가능
	•	🧠 타겟: 다이어트/헬스족, 직장인 점심

⸻

🍚 2. 피마늘 감자볶음밥
	•	감자, 양파, 피마늘, 당근 + 곁들임 계란
	•	👉 장기 보존 가능한 냉동 볶음밥 가능
	•	🧠 타겟: 혼밥족, 기숙사/자취생

⸻

🍱 3. 굴두부 된장보울
	•	굴, 양파, 느타리버섯, 단감 슬라이스 곁들임
	•	👉 전자렌지 보울형 국물 도시락
	•	🧠 타겟: 한식 선호 3040 여성, 시니어

⸻

🥣 4. 레몬 닭가슴살 샐러드
	•	브로콜리, 레몬, 사과, 토마토, 새송이버섯, 오일 드레싱
	•	👉 저지방 저열량, 비건 옵션 가능
	•	🧠 타겟: 운동 전후 간식, 건강식 관심층

⸻

🍲 5. 양파 갈치조림 밀키트
	•	갈치, 양파, 당근, 피마늘, 고구마 (조림용)
	•	👉 냉장 5일/냉동 보존 가능 밀키트 구성
	•	🧠 타겟: 중장년 가족형 식사 구성

⸻

🍧 6. 망고-체리-딸기 냉동 스무디팩
	•	딸기, 체리, 망고, 레몬 조각 → 믹서용 키트
	•	👉 비가열, 냉동 유통 / 1인용 건강 디저트
	•	🧠 타겟: 2030 여성층, 홈카페족

⸻

🍞 7. 단감 & 사과 요거트볼
	•	단감, 사과, 레몬즙, 요거트, 견과
	•	👉 아침식, 간식, 무설탕 디저트
	•	🧠 타겟: 건강 간식, 키즈 간편식

# 1. 키워드 데이터 수급

## 1) 키워드 내 연령별 트렌드 조회

In [3]:
# 월별 데이터 수집 및 병합
def fetch_monthly_data(keywords, start_date, end_date):
    merged_df = pd.DataFrame()
    date_ranges = generate_monthly_dates(start_date, end_date)

    for date_range in tqdm(date_ranges, total=len(date_ranges)):
        # print(f"수집 중: {date_range['start']} ~ {date_range['end']}")
        # API 요청 설정
        category_config = KeywordRequestConfig(
            start_date=date_range['start'],
            end_date=date_range['end'],
            category="50000006",
            keyword=keywords,
            time_unit=TimeUnit.MONTH,
            ages=["20", "30", "40", "50", "60"]
        )
    
        # API 요청
        temp = client.fetch_keyword_age_traffic(category_config)
        merged_df = pd.concat([merged_df, temp])
    
    return merged_df

In [4]:
# 1) 건강식
if not os.path.exists("naverapi_result"):
    os.makedirs("naverapi_result", exist_ok=True)

if not os.path.exists("naverapi_result/건강식_5y_month.parquet"):
    df_health_food = fetch_monthly_data("건강식", "2020-03-01", "2025-03-31")
    df_health_food.to_parquet("naverapi_result/건강식_5y_month.parquet", index=False)
else:
    df_health_food = pd.read_parquet("naverapi_result/건강식_5y_month.parquet")

# 키워드 데이터 시각화
plot_keyword_age_trends(df=df_health_food, word="건강식")

  0%|          | 0/61 [00:00<?, ?it/s]

Loading BokehJS ...

In [5]:
# 2) 다이어트
if not os.path.exists("naverapi_result/다이어트_5y_month.parquet"):
    df_diet = fetch_monthly_data("다이어트", "2020-03-01", "2025-03-31")
    df_diet.to_parquet("naverapi_result/다이어트_5y_month.parquet", index=False)
else:
    df_diet = pd.read_parquet("naverapi_result/다이어트_5y_month.parquet")

# 키워드 데이터 시각화
plot_keyword_age_trends(df=df_diet, word="다이어트")

  0%|          | 0/61 [00:00<?, ?it/s]

Loading BokehJS ...

In [6]:
# 3) 샐러드
if not os.path.exists("naverapi_result/샐러드_5y_month.parquet"):
    df_salad = fetch_monthly_data("샐러드", "2020-03-01", "2025-03-31")
    df_salad.to_parquet("naverapi_result/샐러드_5y_month.parquet", index=False)
else:
    df_salad = pd.read_parquet("naverapi_result/샐러드_5y_month.parquet")

# 키워드 데이터 시각화
plot_keyword_age_trends(df=df_salad, word="샐러드")

  0%|          | 0/61 [00:00<?, ?it/s]

Loading BokehJS ...

In [7]:
# 4) 간편식
if not os.path.exists("naverapi_result/간편식_5y_month.parquet"):
    df_easy_food = fetch_monthly_data("간편식", "2020-03-01", "2025-03-31")
    df_easy_food.to_parquet("naverapi_result/간편식_5y_month.parquet", index=False)
else:
    df_easy_food = pd.read_parquet("naverapi_result/간편식_5y_month.parquet")

# 키워드 데이터 시각화
plot_keyword_age_trends(df=df_easy_food, word="간편식")

  0%|          | 0/61 [00:00<?, ?it/s]

Loading BokehJS ...

원래 프로젝트 방향은 30~40대를 Persona로 설정하여 건강식, 간편식으로 이루어진 요식업 창업이 목적이었으나, 네이버 API 트렌드로는 Persona를 명확히 설정하기가 어려워, 프로젝트의 방향성이 흔들리고 있음 \

추가적으로 장보고 마트 데이터 및 온라인 매출 데이터 EDA를 통하여 다른 방안을 팀 내에서 고려중